In [3]:
# Lib imports
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
# DATASET DIRECTORY CONFIGURATION
train_dir = r"C:\Users\asers\Downloads\Muffin vs Chihuahua\train"
test_dir = r"C:\Users\asers\Downloads\Muffin vs Chihuahua\test"

In [ ]:
# IMAGE PARAMETERS
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
# DATA PREPROCESSING & AUGMENTATION
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [ ]:
# DATA PREPROCESSING & AUGMENTATION
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [ ]:
# RESNET50 

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [9]:
# Configure the model optimizers, loss function, and metrics
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# TRAINING THE RESNET MODEL
# You will notice much higher validation accuracy even in early epochs
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

In [ ]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy with ResNet50: {test_acc}")

In [ ]:
# SAVE THE MODEL
model.save('muffin_vs_chihuahua_resnet.h5')

In [20]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='muffin_vs_chihuahua_resnet.h5'):
    import tensorflow as tf
    from tensorflow.keras.preprocessing import image
    import numpy as np
    from tensorflow.keras.applications.resnet50 import preprocess_input

    # 1. Load the model
    model = tf.keras.models.load_model(model_path)
    
    # 2. Load and preprocess image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))
    
    # 3. Get numerical prediction (probability of class 1)
    pred_prob = model.predict(img_array)[0,0]
    
    # 4. Map the number back to the folder name
    # Since Chihuahua is 0 and Muffin is 1:
    if pred_prob >= 0.5:
        label = "Muffin"      # Index 1
        confidence = pred_prob
    else:
        label = "Chihuahua"   # Index 0
        confidence = 1 - pred_prob # Show how sure it is about being class 0
        
    print(f"Result: {label} (Confidence: {confidence:.2f})")

In [22]:
# Use the 'r' prefix for raw strings to handle Windows backslashes
predict_image(r"C:\Users\asers\Downloads\Muffin vs Chihuahua\final\final1Muffin.png")
predict_image(r"C:\Users\asers\Downloads\Muffin vs Chihuahua\final\final2Chihuahua.png")
predict_image(r"C:\Users\asers\Downloads\Muffin vs Chihuahua\final\final3Muffin.jpg")
predict_image(r"C:\Users\asers\Downloads\Muffin vs Chihuahua\final\final4Muffin.jpg")
predict_image(r"C:\Users\asers\Downloads\Muffin vs Chihuahua\final\final5Chihuahua.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Result: Muffin (Confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Result: Chihuahua (Confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Result: Muffin (Confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Result: Muffin (Confidence: 1.00)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Result: Chihuahua (Confidence: 1.00)
